# Web Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import etree


## Elo Ratings

&#9632; For Fifa Ranking by year <br>
&#9632; Has data on Matches [Data can be grabbed from results.csv] <br> 
<ol>
    &#9633; Home/Away/Neutral <br>
    &#9633; Wins/Losses/Draws <br>
    </ol>

In [2]:
URL = 'https://eloratings.net/'
webpage = requests.get(URL+'2000/')
soup = BeautifulSoup(webpage.text, 'html.parser')
dom = etree.HTML(str(soup))

xpath = '//*[@id="maintable_World"]/div[6]/div/div[1]'
full_xpath = '/html/body/div[1]/div[2]/div[1]/div[6]/div/div[1]'

print(dom.xpath(full_xpath)) # empty list regardless of xpath used

[]



## Transfer Market

&#9632; Squad Size <br>
&#9632; Average Age <br> 
&#9632; Total Value <br>
&#9632; Points <br>
&#9632; Confederation <br>
<ol>
    &#9633; Valuable b/c could get confederation data to use <br>
    &#9633; Potential Issue Confederation stats only, CONACAF not as good as UEFA <br>
    </ol>

In [3]:
URL = 'https://www.transfermarkt.us/wettbewerbe/fifa' # URL of website w/ chart or data
webpage = requests.get(URL) # get webpage from url
soup = BeautifulSoup(webpage.text, 'html.parser') # Calls bs4, gets webpage text
print(soup)

dom = etree.HTML(str(soup))
xpath = '//*[@id="yw1"]/table/tbody/tr[1]'
print(dom.xpath(xpath)) # empty list

<html>
<head><title>404 Not Found</title></head>
<body bgcolor="white">
<center><h1>404 Not Found</h1></center>
<hr/><center>nginx</center>
</body>
</html>

[]



## Fifa Index

&#9632; Get Fifa video game team rankings <br>
<ol>
    &#9633; ATT <br>
    &#9633; MID <br>
    &#9633; DEF <br>
    &#9633; OVR <br>
    &#9633; Team Rating <br>
    </ol>
    
&#9632; How much of this is UNIQUE data? <br>
&#9632; Easier to calculate OVR? <br>

In [21]:
URL = 'https://www.fifaindex.com/teams/?type=1'
webpage = requests.get(URL)
soup = BeautifulSoup(webpage.text, 'html.parser')
dom = etree.HTML(str(soup))
xpath = '/html/body/main/div/div/div[2]/div[2]/table/tbody/tr[3]'
print(dom.xpath(xpath))

[<Element tr at 0x1c36e317508>]



## Fifa Ranking

&#9632; Total Points <br>
&#9632; Ranking <br>

In [20]:
URL = 'https://www.fifa.com/fifa-world-ranking/men?dateId=id13603'
webpage = requests.get(URL)
soup = BeautifulSoup(webpage.text, 'html.parser')
dom = etree.HTML(str(soup))
xpath = '//*[@id="content"]/main/section[2]/div/div/div[1]/table/tbody/tr[1]/td[3]/span[1]'
print(dom.xpath(xpath))


ContentDecodingError: ('Received response with content-encoding: gzip, but failed to decode it.', error('Error -3 while decompressing data: incorrect header check'))